In [1]:
"""Python script that will run the model when invoked via mesa runserver."""
import random
from random import *
import numpy as np
import itertools
from mesa import Agent, Model
from mesa.time import SimultaneousActivation, StagedActivation
from mesa.datacollection import DataCollector
from parameters import infection_table
from mesa.batchrunner import BatchRunner
import pandas

import model
from model import VirusModel
import matplotlib.pyplot as plt

In [2]:
fixed_params = {"run": "NA",
               "init_pop_size": [50, 250, 100, 200]}
variable_params = {"x":[0]}

In [3]:
batch_run = BatchRunner(VirusModel,
                        fixed_parameters=fixed_params,
                        variable_parameters=variable_params,
                        iterations=1,
                        max_steps=10,
                        display_progress=True)

In [4]:
batch_run.run_all()

1it [00:46, 46.01s/it]


In [5]:
agent_data = batch_run.get_collector_agents()
df = agent_data[(0,0)]

In [6]:
df.head()

Species  H13N5  H5N1  H6N2  H9N1  H11N7  H10N9  H1N6  H3N3  \
Step AgentID                                                              
0    1         Human      0     0     0     0      0      0     0     0   
     2         Human      0     0     0     0      0      0     0     0   
     3         Human      0     0     0     0      0      0     0     0   
     4         Human      0     0     0     0      0      0     0     0   
     5         Human      0     0     0     0      0      0     0     0   

              H12N6  ...  H11N5  H2N2  H1N4  H3N1  H10N7  H12N4  H14N7  H16N4  \
Step AgentID         ...                                                        
0    1            0  ...      0     0     0     0      0      0      0      0   
     2            0  ...      0     0     0     0      1      0      0      0   
     3            0  ...      0     0     0     0      0      0      0      0   
     4            0  ...      0     0     0     0      0      0      0      0   
     5            0  ...      0     0     0     0      0      0      0      0   

              H7N1  H8N9  
Step AgentID              
0    1           0     0  
     2           0     0  
     3           0     0  
     4           0     0  
     5           0     0  

[5 rows x 145 columns]